합성 테스트 데이터셋 생성

왜 합성 테스트 데이터(Synthetic Test Dataset) 인가?

RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요합니다.

그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요될 수 있습니다. 또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있습니다.

합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 개발자의 시간을 90% 까지 줄일 수 있습니다.

RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html

In [29]:
!pip install -qU ragas==0.1.20

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


문서 전처리


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

19

In [5]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

데이터셋 생성

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOllama

from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOllama(model="gemma3:1b")
# 데이터셋 비평기
critic_llm = ChatOllama(model="gemma3:1b")
# 문서 임베딩
embeddings = OllamaEmbeddings(model="nomic-embed-text")


ImportError: cannot import name 'MaxRetriesExceeded' from 'ragas.exceptions' (/Users/hyunoozzing/.pyenv/versions/3.11.11/envs/llm/lib/python3.11/site-packages/ragas/exceptions.py)

DocumentStore를 초기화합니다. 사용자 정의 LLM과 임베딩을 사용합니다.

In [ ]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOllama 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOllama(model="gemma3:1b"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)
